In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

In [6]:
train = pd.get_dummies(pd.read_csv('train.csv'), prefix_sep="__").fillna(0)
test = pd.get_dummies(pd.read_csv('test.csv'), prefix_sep="__", )

In [7]:
cat_dummies = [col for col in train 
               if "__" in col]

for col in cat_dummies:
    if col not in test.columns:
        print("Adding missing feature {}".format(col))
        test[col] = 0

test = test.reindex(train.columns, axis=1).drop(['SalePrice'], axis=1).fillna(0)

Adding missing feature Utilities__NoSeWa
Adding missing feature Condition2__RRAe
Adding missing feature Condition2__RRAn
Adding missing feature Condition2__RRNn
Adding missing feature HouseStyle__2.5Fin
Adding missing feature RoofMatl__ClyTile
Adding missing feature RoofMatl__Membran
Adding missing feature RoofMatl__Metal
Adding missing feature RoofMatl__Roll
Adding missing feature Exterior1st__ImStucc
Adding missing feature Exterior1st__Stone
Adding missing feature Exterior2nd__Other
Adding missing feature Heating__Floor
Adding missing feature Heating__OthW
Adding missing feature Electrical__Mix
Adding missing feature GarageQual__Ex
Adding missing feature PoolQC__Fa
Adding missing feature MiscFeature__TenC


In [8]:
train.sample(5)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType__ConLw,SaleType__New,SaleType__Oth,SaleType__WD,SaleCondition__Abnorml,SaleCondition__AdjLand,SaleCondition__Alloca,SaleCondition__Family,SaleCondition__Normal,SaleCondition__Partial
798,799,60,104.0,13518,9,5,2008,2009,860.0,0,...,0,1,0,0,0,0,0,0,0,1
228,229,20,70.0,8521,5,5,1967,1967,0.0,842,...,0,0,0,1,0,0,0,0,1,0
163,164,45,55.0,5500,4,6,1956,1956,0.0,0,...,0,0,0,1,0,0,0,0,1,0
1314,1315,20,60.0,8190,4,6,1954,1954,0.0,732,...,0,0,0,1,0,0,0,0,1,0
993,994,60,68.0,8846,6,5,2005,2006,0.0,0,...,0,1,0,0,0,0,0,0,0,1


In [9]:
test.sample(5)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType__ConLw,SaleType__New,SaleType__Oth,SaleType__WD,SaleCondition__Abnorml,SaleCondition__AdjLand,SaleCondition__Alloca,SaleCondition__Family,SaleCondition__Normal,SaleCondition__Partial
1354,2815,30,0.0,17529,5,5,1924,1950,65.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1426,2887,30,50.0,7288,5,6,1942,1950,0.0,305.0,...,0,0,0,1,0,1,0,0,0,0
118,1579,85,82.0,11105,5,5,1996,1996,0.0,870.0,...,0,0,0,1,0,0,0,0,1,0
862,2323,80,65.0,16219,7,5,2004,2005,0.0,779.0,...,0,0,0,1,0,0,0,0,1,0
1240,2701,60,0.0,9019,6,5,1994,1994,0.0,274.0,...,0,0,0,1,0,0,0,0,1,0


In [10]:
# X_train = np.asarray(train[['LotArea', 'LotFrontage']]).astype(np.float32)
# y_train = np.asarray(train['SalePrice']).astype(np.float32)

#x_values = [i for i in range(11*2)]
X_train = np.array(np.asarray(train.drop(['SalePrice'], axis=1)), dtype=np.float32)

#y_values = [2*i + 1 for i in range(11)]
y_train = np.array(np.asarray(train['SalePrice'].fillna(0)), dtype=np.float32)


In [12]:
X_train.shape

(1460, 289)

In [13]:
y_train.shape

(1460,)

In [142]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [167]:
reg = RandomForestRegressor(max_depth=200, n_estimators=5000, n_jobs=-1)
reg.fit(X_train, y_train)


0.9815120940855117

In [ ]:
reg.score(X_train, y_train)

In [168]:
cross_val_score(reg, X_train, y_train, cv=3)

array([0.88695021, 0.85065336, 0.82462337])

In [164]:
# output = train.copy()
# output['SalePrice_pred'] = reg.predict(output.drop(['SalePrice'], axis=1))
# output[['SalePrice', 'SalePrice_pred']].sample(15)

In [169]:
output = test.copy()

output['SalePrice'] = reg.predict(output)
output[['Id', 'SalePrice']].to_csv('my_submission.csv', index=False)

In [292]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(289, 144)
        self.fc2 = nn.Linear(144, 72)
        self.fc3 = nn.Linear(72, 36)
        self.fc4 = nn.Linear(36, 18)
        self.fc5 = nn.Linear(18, 1)

    def forward(self, x):
        x = (self.fc1(x))
        x = (self.fc2(x))
        x = (self.fc3(x))
        x = (self.fc4(x))
        x = (self.fc5(x))
        return x
    


In [299]:
net = Net()
net(torch.from_numpy(X_train))

tensor([[-41.2136],
        [-38.0575],
        [-46.4659],
        ...,
        [-58.9491],
        [-45.7229],
        [-46.5732]], grad_fn=<AddmmBackward>)

In [294]:
import torch.optim as optim


# optimizer = optim.Adam(net.parameters(), lr=0.01)
# loss_function = nn.MSELoss()

loss_function = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)



In [295]:
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)

# scale units
X_max, _ = torch.max(X_train_tensor, 0)
y_max, _ = torch.max(y_train_tensor, 0)

X_train_norm = torch.div(X_train_tensor, X_max)
y_train_norm = torch.div(y_train_tensor, y_max)

y_train_norm

tensor([0.2762, 0.2404, 0.2960,  ..., 0.3530, 0.1882, 0.1954])

In [296]:
BATCH_SIZE = 5
EPOCHS = 50

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(X_train_norm), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        batch_X = (X_train_norm[i:i+BATCH_SIZE]).view(-1, 1, 289)
        batch_y = (y_train_norm[i:i+BATCH_SIZE])

        net.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update
        
    print('epoch {}, loss {}'.format(epoch, loss.item()))


 29%|██▉       | 86/292 [00:00<00:00, 854.50it/s]

epoch 0, loss 0.004516032990068197


 29%|██▉       | 86/292 [00:00<00:00, 859.89it/s]

epoch 1, loss 0.004474537447094917


 26%|██▋       | 77/292 [00:00<00:00, 769.26it/s]

...

epoch 47, loss 0.004003890790045261


 27%|██▋       | 78/292 [00:00<00:00, 777.14it/s]

epoch 48, loss 0.004002673085778952


100%|██████████| 292/292 [00:00<00:00, 784.91it/s]

epoch 49, loss 0.004001527093350887


In [300]:
X_test_tensor = torch.from_numpy(np.asarray(test, dtype=np.float32))
X_test_norm = torch.div(X_test_tensor, X_max)

yHat = net(X_test_norm)
yHat

tensor([[0.1049],
        [0.1122],
        [0.0972],
        ...,
        [0.1027],
        [0.0819],
        [0.0710]], grad_fn=<AddmmBackward>)

In [301]:
torch.mul(yHat, y_max)

tensor([[79223.8438],
        [84727.7734],
        [73371.7812],
        ...,
        [77505.0000],
        [61857.6016],
        [53626.7695]], grad_fn=<MulBackward0>)

In [302]:
output = test.copy()

output['SalePrice'] = torch.mul(yHat, y_max).detach().numpy()
output[['Id', 'SalePrice']].to_csv('my_submission_nn.csv', index=False)

In [303]:
output[['Id', 'SalePrice']].head(10)

,Id,SalePrice
0,1461,79223.843750
1,1462,84727.773438
2,1463,73371.781250
3,1464,80335.687500
4,1465,89928.632812
5,1466,69237.492188
6,1467,73931.835938
7,1468,78886.625000
8,1469,72698.445312
9,1470,79032.671875
